In [2]:
!pip install emoji

In [1]:
import emoji

In [4]:
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

❤️
⚾
😃
😞
🍴
👅
🕶
💏
:poop:


In [3]:
emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":grinning_face_with_big_eyes:",
                    "3": ":disappointed_face:",
                    "4": ":fork_and_knife:",
                    "5": ":tongue:",
                    "6": ":sunglasses:",
                    "7": ":kiss:",
                    "8": ":poop:",
                   }

In [5]:
import pandas as pd
import numpy as np

In [6]:
train=pd.read_csv("./dataset/train_emoji.csv",header=None)

In [7]:
test=pd.read_csv("./dataset/test_emoji.csv",header=None)

In [8]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [9]:
X_train=train[0]
Y_train=train[1]
X_test=test[0]
Y_test=test[1]

In [10]:
for i in range(5):
    print(X_train[i],emoji.emojize(emoji_dictionary[str(Y_train[i])]))

never talk to me again 😞
I am proud of your achievements 😃
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴


In [11]:
f=open('./dataset/glove.6B.50d.txt')

In [12]:
embeddings_index={}
for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype="float")
    embeddings_index[word]=coefs
f.close()  

In [13]:
X_train.shape

(132,)

In [14]:
def embedding_output(X):
    maxLen=10
    emb_dim=50
    embedding_op=np.zeros((X.shape[0],maxLen,50))
    
    for ix in range(X.shape[0]):
        X[ix]=X[ix].split()
        for ij in range(len(X[ix])):
            try:
                embedding_op[ix][ij]=embeddings_index[X[ix][ij].lower()]
            except:
                embedding_op[ix][ij]=np.zeros((50,))
    return embedding_op

In [15]:
emb_matrix_train=embedding_output(X_train)
emb_matrix_test=embedding_output(X_test)

/home/shrey/.local/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [16]:
emb_matrix_train.shape

(132, 10, 50)

In [20]:
from keras.models import Sequential
from keras.layers import *
from keras.utils import to_categorical

In [23]:
Y_train.shape

(132,)

In [24]:
from keras.utils import to_categorical

In [25]:
Y_train=to_categorical(Y_train,num_classes=5)
Y_test=to_categorical(Y_test,num_classes=5)

DEFINING MODEL

In [43]:
model=Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64,return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 10, 64)            29440     
_________________________________________________________________
dropout_6 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_4 (Activation)    (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
__________________________________________________

In [44]:
from keras.callbacks import EarlyStopping,ModelCheckpoint

checkpoint=ModelCheckpoint("best_model.h5",monitor='val_loss',verbose=True,save_best_only=True)
earlystop=EarlyStopping(monitor='val_accuracy',patience=10)
hist=model.fit(emb_matrix_train,Y_train,epochs=100,batch_size=64,shuffle=True,validation_split=0.2)

Train on 105 samples, validate on 27 samples
Epoch 1/100
105/105 [==============================] - 1s 5ms/step - loss: 1.5949 - accuracy: 0.2762 - val_loss: 1.5999 - val_accuracy: 0.1852
Epoch 2/100
105/105 [==============================] - 0s 204us/step - loss: 1.5687 - accuracy: 0.2667 - val_loss: 1.5996 - val_accuracy: 0.2593
Epoch 3/100
105/105 [==============================] - 0s 249us/step - loss: 1.5361 - accuracy: 0.3429 - val_loss: 1.6080 - val_accuracy: 0.2222
Epoch 4/100
105/105 [==============================] - 0s 232us/step - loss: 1.5104 - accuracy: 0.3333 - val_loss: 1.6178 - val_accuracy: 0.2222
Epoch 5/100
105/105 [==============================] - 0s 275us/step - loss: 1.4748 - accuracy: 0.2952 - val_loss: 1.6337 - val_accuracy: 0.2222
Epoch 6/100
105/105 [==============================] - 0s 276us/step - loss: 1.4788 - accuracy: 0.3143 - val_loss: 1.6452 - val_accuracy: 0.2222
Epoch 7/100
105/105 [==============================] - 0s 239us/step - loss: 1.4703 - a

Epoch 57/100
105/105 [==============================] - 0s 253us/step - loss: 0.0751 - accuracy: 0.9714 - val_loss: 1.6229 - val_accuracy: 0.6296
Epoch 58/100
105/105 [==============================] - 0s 252us/step - loss: 0.0889 - accuracy: 0.9714 - val_loss: 1.5928 - val_accuracy: 0.6296
Epoch 59/100
105/105 [==============================] - 0s 241us/step - loss: 0.0550 - accuracy: 1.0000 - val_loss: 1.4694 - val_accuracy: 0.6296
Epoch 60/100
105/105 [==============================] - 0s 232us/step - loss: 0.0688 - accuracy: 0.9714 - val_loss: 1.5659 - val_accuracy: 0.6296
Epoch 61/100
105/105 [==============================] - 0s 237us/step - loss: 0.0663 - accuracy: 0.9905 - val_loss: 1.8662 - val_accuracy: 0.5926
Epoch 62/100
105/105 [==============================] - 0s 264us/step - loss: 0.0826 - accuracy: 0.9810 - val_loss: 1.9855 - val_accuracy: 0.5926
Epoch 63/100
105/105 [==============================] - 0s 250us/step - loss: 0.1196 - accuracy: 0.9714 - val_loss: 1.8569 -

In [50]:
pred=model.predict_classes(emb_matrix_test)

In [51]:
print(pred)

[4 3 2 2 2 2 1 2 4 2 1 2 0 3 1 3 2 2 1 2 0 0 4 2 3 1 2 0 1 2 0 1 2 2 0 2 2
 4 1 2 1 0 0 2 2 0 2 2 2 1 3 0 3 2 2 4]


In [52]:
model.evaluate(emb_matrix_test,Y_test)

56/56 [==============================] - 0s 213us/step


[2.4341043744768416, 0.5714285969734192]

In [54]:
for i in range(30):
    print(' '.join(X_test[i]))
    print(emoji.emojize(emoji_dictionary[str(np.argmax(Y_test[i]))]))
    print(emoji.emojize(emoji_dictionary[str(pred[i])]))

I want to eat
🍴
🍴
he did not answer
😞
😞
he got a raise
😃
😃
she got me a present
❤️
😃
ha ha ha it was so funny
😃
😃
he is a good friend
❤️
😃
I am upset
❤️
⚾
We had such a lovely dinner tonight
❤️
😃
where is the food
🍴
🍴
Stop making this joke ha ha ha
😃
😃
where is the ball
⚾
⚾
work is hard
😞
😃
This girl is messing with me
😞
❤️
are you serious ha ha
😃
😞
Let us go play baseball
⚾
⚾
This stupid grader is not working
😞
😞
work is horrible
😞
😃
Congratulation for having a baby
😃
😃
stop messing around
😞
⚾
any suggestions for dinner
🍴
😃
I love taking breaks
❤️
❤️
you brighten my day
😃
❤️
I boiled rice
🍴
🍴
she is a bully
😞
😃
Why are you feeling bad
😞
😞
I am upset
😞
⚾
I worked during my birthday
😞
😃
My grandmother is the love of my life
❤️
❤️
enjoy your break
😃
⚾
valentine day is near
❤️
😃


In [68]:
em_inp=embedding_output(new_txt)

TypeError: data type not understood

In [70]:
X_test

0                                 [I, want, to, eat]
1                             [he, did, not, answer]
2                                [he, got, a, raise]
3                         [she, got, me, a, present]
4                   [ha, ha, ha, it, was, so, funny]
5                          [he, is, a, good, friend]
6                                     [I, am, upset]
7        [We, had, such, a, lovely, dinner, tonight]
8                             [where, is, the, food]
9             [Stop, making, this, joke, ha, ha, ha]
10                            [where, is, the, ball]
11                                  [work, is, hard]
12               [This, girl, is, messing, with, me]
13                       [are, you, serious, ha, ha]
14                     [Let, us, go, play, baseball]
15          [This, stupid, grader, is, not, working]
16                              [work, is, horrible]
17            [Congratulation, for, having, a, baby]
18                           [stop, messing, a